# Invoke-WebRequest
**Description**: Scrape data from a webpage

**Website**: [Quotes to Scrape](quotes.toscrape.com)

<img src="C:\Users\dkfc06bf\Documents\GitHub\PowerShell\Invoke-WebRequest\screencapture-quotes-toscrape-2020-06-12-02_16_10-edit.png" width="50%">

## Step 1 - Get and inspect the data


## Step 2 - 
